In [1]:
%run MasterNetwork.py

How many mini grids do you want to simulate?  6
How many houses do you want in MiniGrid 1?  12
Average sunlight hours for MiniGrid 1?  7
Enter average daily consumption for houses in MiniGrid 1 (in kWh):  33
How many houses do you want in MiniGrid 2?  9
Average sunlight hours for MiniGrid 2?  5
Enter average daily consumption for houses in MiniGrid 2 (in kWh):  38
How many houses do you want in MiniGrid 3?  7
Average sunlight hours for MiniGrid 3?  6
Enter average daily consumption for houses in MiniGrid 3 (in kWh):  30
How many houses do you want in MiniGrid 4?  16
Average sunlight hours for MiniGrid 4?  7
Enter average daily consumption for houses in MiniGrid 4 (in kWh):  28
How many houses do you want in MiniGrid 5?  8
Average sunlight hours for MiniGrid 5?  8
Enter average daily consumption for houses in MiniGrid 5 (in kWh):  22
How many houses do you want in MiniGrid 6?  5
Average sunlight hours for MiniGrid 6?  6
Enter average daily consumption for houses in MiniGrid 6 (in kWh): 

Total revenue from selling to houses: $739.963
Total unmet demand: 0 kWh

MiniGrid 1:
	Number of houses: 12
	Average consumption/hour: 1.65 kWh
	Solar capacity: 84.574 kWp
	Battery capacity: 493.35 kWh

MiniGrid 2:
	Number of houses: 9
	Average consumption/hour: 1.9 kWh
	Solar capacity: 102.258 kWp
	Battery capacity: 426.075 kWh

MiniGrid 3:
	Number of houses: 7
	Average consumption/hour: 1.5 kWh
	Solar capacity: 52.325 kWp
	Battery capacity: 261.625 kWh

MiniGrid 4:
	Number of houses: 16
	Average consumption/hour: 1.4 kWh
	Solar capacity: 95.68 kWp
	Battery capacity: 558.133 kWh

MiniGrid 5:
	Number of houses: 8
	Average consumption/hour: 1.1 kWh
	Solar capacity: 32.89 kWp
	Battery capacity: 219.267 kWh

MiniGrid 6:
	Number of houses: 5
	Average consumption/hour: 1.25 kWh
	Solar capacity: 31.146 kWp
	Battery capacity: 155.729 kWh


In [2]:
import pandas as pd

In [3]:
grid1 = pd.read_csv('mini_grid_log_1.csv')

In [17]:
grid1['total_demand_kWh']

0    3.00
1    2.75
2    2.50
3    2.25
4    2.00
5    1.75
6    1.50
7    1.75
8    2.00
9    2.25
Name: total_demand_kWh, dtype: float64

In [28]:
grid1.loc[:2]

,simulation_hour,generation_kWh,total_demand_kWh,unmet_demand_kWh,internal_grid_transactions_kWh,external_grid_transactions_kWh,revenue_USD,battery_level_kWh,battery_%
0,0,0.0,3.00,0,[],[],0.150,109.125,29.20
1,1,0.0,2.75,0,[],[],0.288,106.375,28.46
2,2,0.0,2.50,0,[],[],0.413,103.875,27.79


In [31]:
grid1.loc[:2,['total_demand_kWh', 'generation_kWh']]

,total_demand_kWh,generation_kWh
0,3.00,0.0
1,2.75,0.0
2,2.50,0.0


In [65]:
grid1.iloc[:8,1:3]

,generation_kWh,total_demand_kWh
0,0.0,3.00
1,0.0,2.75
2,0.0,2.50
3,0.0,2.25
4,0.0,2.00
5,0.0,1.75
6,0.0,1.50
7,0.0,1.75


In [95]:
grid1['total_demand_kWh'].iloc[0:4].values, grid1['total_demand_kWh'].iloc[1:5].values, grid1['total_demand_kWh'].iloc[2:6].values

(array([3.  , 2.75, 2.5 , 2.25]),
 array([2.75, 2.5 , 2.25, 2.  ]),
 array([2.5 , 2.25, 2.  , 1.75]))

In [96]:
np.array(grid1['total_demand_kWh'].iloc[4]), np.array(grid1['total_demand_kWh'].iloc[5]), np.array(grid1['total_demand_kWh'].iloc[6])

(array(2.), array(1.75), array(1.5))

In [50]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Function to create dataset for time-series prediction
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Load the dataset
data1 = pd.read_csv('mini_grid_log_1.csv')
data2 = pd.read_csv('mini_grid_log_2.csv')
data3 = pd.read_csv('mini_grid_log_3.csv')

# Concatenate the datasets
data = pd.concat([data1, data2, data3], ignore_index=True)

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['total_demand_kWh']])

# Define the number of time steps (i.e., the number of hours of demand data to use as input for each prediction)
time_steps = 24

# Create the dataset for supervised learning
X, y = create_dataset(pd.DataFrame(scaled_data), pd.DataFrame(scaled_data), time_steps)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [107]:
scaled_data.shape

(26280, 1)

In [106]:
data[['total_demand_kWh']].shape

(26280, 1)

In [110]:
X_train.shape, X_test.shape

((21004, 24, 1), (5252, 24, 1))

In [109]:
y_train.shape

(21004, 1)

In [ ]:
for running the simulation we have this snippet -             
if __name__ == "__main__":
    # Example demand profile (24 values representing hourly consumption)
    demand_profile = [1.2, 1.1, 1.0, 0.9, 0.8, 0.7, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2,
                  1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 1.8, 1.7, 1.6, 1.5, 1.3, 1.2]


    # Create a few houses with random average daily consumption
    # (id, base_demand, demand_profile)   random.uniform(5, 6)
    houses1 = [House(i, 12/24, demand_profile) for i in range(5)]
    houses2 = [House(i,  10/24, demand_profile) for i in range(9)]
    houses3 = [House(i, 11/24, demand_profile) for i in range(2)]

    
    
    # Create a ConventionalGrid with certain prices
    # (buying_price, selling_price)
    conventional_grid = ConventionalGrid(0.03, 0.06)

    # Create MiniGrid instances without setting neighboring_grids yet
    minigrid1 = MiniGrid(id=1, houses = houses1, avg_sunlight_hours = 8, selling_price = 0.05, num_days_backup = 5, 
                         neighboring_grids = [], conventional_grid=conventional_grid, safety_factor = 1.5)
    
    minigrid2 = MiniGrid(id=2, houses = houses2, avg_sunlight_hours = 5, selling_price = 0.05, num_days_backup = 5, 
                         neighboring_grids = [], conventional_grid=conventional_grid, safety_factor = 1.5)
    
    minigrid3 = MiniGrid(id=3, houses = houses3, avg_sunlight_hours = 6, selling_price = 0.05, num_days_backup = 5, 
                         neighboring_grids = [], conventional_grid=conventional_grid, safety_factor = 1.5)
    

    # Now that all MiniGrid instances are created, set the neighboring grids
    minigrid1.neighboring_grids = [minigrid2, minigrid3]
    minigrid2.neighboring_grids = [minigrid3, minigrid1]
    minigrid3.neighboring_grids = [minigrid1, minigrid2]
    
    # Assign houses to the respective grids and initialize CSVs
    for house in houses1:
        house.assign_to_grid(minigrid1)
       # house.initialize_csv()
    
    for house in houses2:
        house.assign_to_grid(minigrid2)
        #house.initialize_csv()
    
    for house in houses3:
        house.assign_to_grid(minigrid3)
        #house.initialize_csv()
   
    # Add the MiniGrids to a list
    minigrids = [minigrid1, minigrid2, minigrid3]

    # Create a Simulation instance
    # (conventional_grid, grids)
    simulation = Simulation(conventional_grid, minigrids)

    # Simulate for a certain number of days
    simulation.simulate_days(7)
            
    
    # Save the logs
    simulation.save_logs()
    for grid in simulation.grids:
        grid.save_logs()        


    # Output some results
    print(f"Total revenue from selling to houses: ${round(simulation.total_client_revenue, 3)}")
#    print(f"Total revenue from grid transactions: ${simulation.revenue}")
    print(f"Total unmet demand: {round(simulation.unmet_demand, 3)} kWh")
    

    # Output system information
    for i, grid in enumerate(simulation.grids):
        # Calculate average consumption for each house in the minigrid
        total_consumption = sum(house.current_demand for house in grid.houses)
        average_consumption = total_consumption / len(grid.houses) if grid.houses else 0

        print(f"\nMiniGrid {i + 1}:")
        print(f"\tNumber of houses: {len(grid.houses)}")
        print(f"\tAverage consumption/hour: {round(average_consumption, 3)} kWh")
        print(f"\tSolar capacity: {round(grid.solar_pv.capacity, 3)} kWp")
        print(f"\tBattery capacity: {round(grid.battery.capacity, 3)} kWh")
            
       